In [21]:
import numpy as np
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [22]:
N=300
x,t=datasets.make_moons(N,noise=0.3)
t=t.reshape(N,1)
x_train,x_test,t_train,t_test=train_test_split(x,t,test_size=0.2)

In [23]:
class MLP(Model):
    
    def __init__(self,hidden_dim,output_dim):
        super().__init__()
        self.l1=Dense(hidden_dim,activation='sigmoid')
        self.l2=Dense(output_dim,activation='sigmoid')
        
    def call(self,x):
        h=self.l1(x)
        y=self.l2(h)
        return y

model=MLP(3,1)

In [24]:
criterion=losses.BinaryCrossentropy()
optimizer=optimizers.SGD(learning_rate=0.1)

def compute_loss(t,y):
    return criterion(t,y)

def train_step(x,t):
    with tf.GradientTape() as tape:
        preds=model(x)
        loss=compute_loss(t,preds)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    return loss

In [25]:
epoches=100
batch_size=10
n_batches=x_train.shape[0]//batch_size

for epoch in range(epoches):
    train_loss=0.
    x_,t_=shuffle(x_train,t_train)
    
    for batch in range(n_batches):
        start=batch*batch_size
        end=start+batch_size
        loss=train_step(x_[start:end],t_[start:end])
        train_loss+=loss.numpy()
    
    print('epoch:{} , loss:{:.3}'.format(epoch+1,train_loss))
    


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch:1 , loss:17.1
epoch:2 , loss:16.0
epoch:3 , loss:15.0
epoch:4 , loss:14.2
epoch:5 , loss:13.5
epoch:6 , loss:12.9
epoch:7 , loss:12.3
epoch:8 , loss:11.8
epoch:9 , loss:11.4
epoch:10 , loss:11.1
epoch:11 , loss:10.8
epoch:12 , loss:10.5
epoch:13 , loss:10.3
epoch:14 , loss:10.1
epoch:15 , loss:9.9
epoch:16 , loss:9.77
epoch:17 , loss:9.6
epoch:18 , loss:9.49
epoch:19 , loss:9.36
epoch:20 , loss:9.28
epoch:21 , loss:9.18
epoch:22 , loss:9.11
epoch:23 , loss:9.03
epoch:24 , loss:8.97
epoch:25 , loss

In [26]:
test_loss=metrics.Mean()
test_acc=metrics.BinaryAccuracy()
def test_step(x,t):
    preds=model(x)
    loss=compute_loss(t,preds)
    test_loss(loss)
    test_acc(t,preds)
    return preds
test_step(x_test,t_test)
print('test_loss: {:.3f}, test_acc: {:.3f}'.format(test_loss.result(),test_acc.result()))

test_loss: 0.410, test_acc: 0.850
